Cours : [DLC] SQL pour Débutant(e)s Jour 6 - Les Outils Avancés 
Lien : https://www.youtube.com/watch?v=mc4zOX1j7_A&list=PLyh35eYRez8cAEBfi1LWN70Cs77bBvCOp&index=6

Github le coin stat : https://github.com/LeCoinStat/LeCoinStat/tree/main/Bien_Debuter_SQL
Fichiers Github récupérés : https://github.com/LeCoinStat/LeCoinStat/tree/main/Bien_Debuter_SQL/Jour6

In [1]:
import duckdb
from skimpy import skim

In [2]:
# Connection à la librairie duckdb
conn = duckdb.connect()

## Fichier des ventes

In [3]:
# Requête SQL : récupération de toutes les données du fichier .csv en DF
sales_df = conn.execute(
    """
    SELECT *
    FROM read_csv_auto('data/indexE/Ventes.csv', header=True)
    """
).df()

In [4]:
# Stats de la DF
skim(sales_df)

╭──────────────────────────────────────────────── skimpy summary ─────────────────────────────────────────────────╮
│          Data Summary                Data Types                                                                 │
│ ┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓ ┏━━━━━━━━━━━━━┳━━━━━━━┓                                                          │
│ ┃ dataframe         ┃ Values ┃ ┃ Column Type ┃ Count ┃                                                          │
│ ┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩ ┡━━━━━━━━━━━━━╇━━━━━━━┩                                                          │
│ │ Number of rows    │ 100    │ │ int32       │ 5     │                                                          │
│ │ Number of columns │ 7      │ │ datetime64  │ 1     │                                                          │
│ └───────────────────┴────────┘ │ float64     │ 1     │                                                          │
│                                └─────────────┴───────┘                                                          │
│                                                     number                                                      │
│ ┏━━━━━━━━━━━━━━━━━━━━┳━━━━━┳━━━━━━━━┳━━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━━┳━━━━━━━━┳━━━━━━━━━┓  │
│ ┃ column_name        ┃ NA  ┃ NA %   ┃ mean    ┃ sd     ┃ p0     ┃ p25    ┃ p50   ┃ p75    ┃ p100   ┃ hist    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━━━╇━━━━━╇━━━━━━━━╇━━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━━╇━━━━━━━━╇━━━━━━━━━┩  │
│ │ VenteID            │   0 │      0 │    50.5 │  29.01 │      1 │  25.75 │  50.5 │  75.25 │    100 │ ▇▇▇▇▇▇  │  │
│ │ ClientID           │   0 │      0 │   51.26 │  29.18 │      1 │  27.75 │  53.5 │  77.25 │     99 │ ▆▇▅▇▆▇  │  │
│ │ EmployeID          │   0 │      0 │   50.65 │   29.4 │      2 │     23 │  49.5 │  76.25 │    100 │ ▇▇▇▆▇▇  │  │
│ │ ProduitID          │   0 │      0 │   48.08 │  24.56 │      3 │     29 │    46 │  65.25 │     98 │ ▅▆▇▇▅▃  │  │
│ │ QuantiteVendue     │   0 │      0 │   48.36 │  30.17 │      1 │     22 │  50.5 │     71 │    100 │ ▇▇▃▇▃▇  │  │
│ │ MontantTotal       │   0 │      0 │    8278 │  11580 │  59.98 │   1542 │  2979 │   9249 │  48500 │ ▇▁▁  ▁  │  │
│ └────────────────────┴─────┴────────┴─────────┴────────┴────────┴────────┴───────┴────────┴────────┴─────────┘  │
│                                                    datetime                                                     │
│ ┏━━━━━━━━━━━━━━━━━━┳━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┓  │
│ ┃ column_name      ┃ NA   ┃ NA %    ┃ first                      ┃ last                       ┃ frequency    ┃  │
│ ┡━━━━━━━━━━━━━━━━━━╇━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━┩  │
│ │ DateVente        │    0 │       0 │    2020-01-08 12:43:33     │    2023-12-27 15:39:42     │ None         │  │
│ └──────────────────┴──────┴─────────┴────────────────────────────┴────────────────────────────┴──────────────┘  │
╰────────────────────────────────────────────────────── End ──────────────────────────────────────────────────────╯

In [5]:
# Enregistrement de la DF en BD virtuelle (requête SQL : VIEW)
conn.register(view_name='sales_db', python_object=sales_df)

## Requêtes SQL

#### Créer une colonne sur l'évolution du CA année par année par taux de croissance

1ère étape : calculer le CA par année

In [17]:
conn.execute(
    """
    -- Colonnes à récupérer
    SELECT
        YEAR(s.DateVente) AS Annee
        , ROUND(SUM(s.MontantTotal), 0) AS "Ventes annuelles"
    -- BD à récupérer
    FROM sales_db s
    -- TCD
    GROUP BY Annee
    -- Trie
    ORDER BY Annee ASC
    """
).df()

,Annee,Ventes annuelles
0,2020,210899.0
1,2021,219847.0
2,2022,279517.0
3,2023,117509.0


2ème étape : évolution du CA par année

In [22]:
conn.execute(
    """
    -- Colonnes à récupérer
    SELECT
        YEAR(s.DateVente) AS Annee
        , ROUND(SUM(s.MontantTotal), 0) AS "Ventes annuelles"
        -- Valeur ligne - valeur ligne précédente
        , ROUND(SUM(s.MontantTotal), 0) - LAG(ROUND(SUM(s.MontantTotal), 0))
        -- Fonctions de fenêtrage
        OVER(
            -- Trie
            ORDER BY YEAR(s.DateVente) ASC
        ) AS Variation
    -- BD à récupérer
    FROM sales_db s
    -- TCD
    GROUP BY Annee
    -- Trie
    ORDER BY Annee ASC
    """
).df()

,Annee,Ventes annuelles,Variation
0,2020,210899.0,NaN
1,2021,219847.0,8948.0
2,2022,279517.0,59670.0
3,2023,117509.0,-162008.0


3ème étape : détermination du taux de croissance

In [49]:
conn.execute(
    """
    -- Colonnes à récupérer
    SELECT
        YEAR(s.DateVente) AS Annee
        , ROUND(SUM(s.MontantTotal), 0) AS "Ventes annuelles"
        , COALESCE(
            -- Numérateur
            (
                -- Valeur de la ligne en cours
                ROUND(SUM(MontantTotal), 0)
                -- Valeur de la ligne précédente 
                - LAG(ROUND(SUM(MontantTotal), 0)) OVER(ORDER BY YEAR(DateVente))
            ) 
            -- Dénominateur
            / LAG(ROUND(SUM(MontantTotal), 0)) OVER(ORDER BY YEAR(DateVente))
            , 0) * 100
            AS "Taux de croissance en %"
    -- BD à récupérer
    FROM sales_db s
    -- TCD
    GROUP BY Annee
    -- Trie
    ORDER BY Annee ASC
    """
).df()

,Annee,Ventes annuelles,Taux de croissance en %
0,2020,210899.0,0.000000
1,2021,219847.0,4.242789
2,2022,279517.0,27.141603
3,2023,117509.0,-57.959981
